## **Installing dependencies and clonning repository**

In [1]:
!git clone https://github.com/danilkos00/PyTorch-SFT-and-RL.git -qq
%cd PyTorch-SFT-and-RL
!pip install vllm -qq

/kaggle/working/PyTorch-SFT-and-RL
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 4.4 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 82.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 14.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Load the fine-tuned model

In [2]:
import kagglehub


model_path = kagglehub.model_download("danilkos0101/qwen-1.5b-gsm8k-grpo_v3/pyTorch/default")

## Load the GSM8K dataset


In [3]:
from data.dataset import get_sft_dataset

sft_dataset = get_sft_dataset()

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

## Overwriting config for grpo training

In [6]:
%%writefile ./config/grpo_config.json
{
  "optimizer": {
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "betas": [0.9, 0.95]
  },
  "training": {
    "loss_type": "reinforce_with_baseline",
    "tokenizer_name": "Qwen/Qwen2.5-Math-1.5B",
    "use_std_normalization": 1,
    "epochs_per_rollout_batch": 1,
    "n_grpo_steps": 200,
    "rollout_batch_size": 256,
    "group_size": 8,
    "train_batch_size": 256,
    "gradient_accumulation_steps": 128,
    "gpu_memory_utilization": 0.35,
    "sampling_temperature": 0.5,
    "sampling_max_tokens": 256,
    "sampling_min_tokens": 4,
    "advantage_eps": 1e-6,
    "grad_clip": 1.0,
    "checkpoint_path": "checkpoint.tar"
  }
}

Overwriting ./config/grpo_config.json


## Run GRPO training algorithm

In [ ]:
from training_utils.training import grpo_train


grpo_train(
    dataset=sft_dataset,
    config_path='./config/grpo_config.json',
    model_weights_path=model_path,
)

## Init vllm instance

In [ ]:
from vllm import LLM, SamplingParams


llm = LLM(
    model=model_path,
    tokenizer='Qwen/Qwen2.5-Math-1.5B',
    tensor_parallel_size=1,
    dtype='float16',
    gpu_memory_utilization=0.5,
    swap_space=4,
    enforce_eager=True
)

## Run Inferece on some GSM8K questions

In [16]:
prompts, responses = sft_dataset['test'][50:55].values()

sampling_params = SamplingParams(
    temperature=0.5,
    max_tokens=256,
    min_tokens=4,
    stop=['</answer>'],
    include_stop_str_in_output=True
)

outputs = llm.generate(prompts, sampling_params)

for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}\n")
    print(f"Generated text: {generated_text!r}\n")
    print(f"Ground truth response: {responses[i]!r}")
    print("-"*50)

Adding requests:   0%|          | 0/5 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/5 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: 'Question: Lloyd has an egg farm. His chickens produce 252 eggs per day and he sells them for $2 per dozen. How much does Lloyd make on eggs per week?Answer: <think>'

Generated text: "There are 7 days in a week, so Lloyd's chickens produce 252 eggs * 7 days = <<252*7=1764>>1764 eggs per week.\nSince there are 12 eggs in a dozen, Lloyd makes 1764 eggs / 12 eggs/dozen = <<1764/12=147>>147 dozens of eggs per week.\nSince each dozen of eggs is sold for $2, Lloyd makes 147 dozens * $2/dozen = $<<147*2=294>>294 per week.</think> <answer>#### 294</answer>"

Ground truth response: "In a week, Lloyd's egg farm produces 252 x 7 = <<252*7=1764>>1764 eggs.\nThere are 1764/12 = <<1764/12=147>>147 dozens of eggs produced in a week.\nSo, Lloyd makes 147 x $2 = $<<147*2=294>>294 on eggs.</think> <answer>#### 294</answer>"
--------------------------------------------------
Prompt: "Question: Tom's ship can travel at 10 miles per hour.  He is sailing from 1 to 4 PM.  He then travels back at a r

## Compute answer accuracy on the test dataset

In [6]:
import re

sampling_params = SamplingParams(
    temperature=0.5,
    max_tokens=256,
    min_tokens=4,
    stop=['</answer>'],
    include_stop_str_in_output=True
)

prompts, responses = sft_dataset['test'][:].values()

outputs = llm.generate(prompts, sampling_params)

n_true_answers = 0
n = 0

for output, gt_response in zip(outputs, responses):
    n += 1
    prompt = output.prompt
    generated_text = output.outputs[0].text

    generated_answer = re.search(r'<answer>#### (.+)</answer>', generated_text)

    if generated_answer is None:
        continue

    generated_answer = generated_answer.group(1)
    gt_answer = re.search(r'<answer>#### (.+)</answer>', gt_response).group(1)

    if generated_answer == gt_answer:
        n_true_answers += 1
        
print(f'Accuracy on the test dataset: {n_true_answers / len(prompts):.2f}')

Adding requests:   0%|          | 0/1319 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1319 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Accuracy on the test dataset: 0.74
